<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#10-1-二部マッチング" data-toc-modified-id="10-1-二部マッチング-1">10-1 二部マッチング</a></span><ul class="toc-item"><li><span><a href="#リスト10-1-増加道を使った二部マッチングを求めるアルゴリズム" data-toc-modified-id="リスト10-1-増加道を使った二部マッチングを求めるアルゴリズム-1.1">リスト10-1 増加道を使った二部マッチングを求めるアルゴリズム</a></span></li></ul></li><li><span><a href="#10-2-辺素な道" data-toc-modified-id="10-2-辺素な道-2">10-2 辺素な道</a></span><ul class="toc-item"><li><span><a href="#リスト10-2-増加道を見つけるための、ラベリング法を使った辺素な道の数え上げ" data-toc-modified-id="リスト10-2-増加道を見つけるための、ラベリング法を使った辺素な道の数え上げ-2.1">リスト10-2 増加道を見つけるための、ラベリング法を使った辺素な道の数え上げ</a></span></li></ul></li><li><span><a href="#10-3-最大フロー" data-toc-modified-id="10-3-最大フロー-3">10-3 最大フロー</a></span><ul class="toc-item"><li><span><a href="#リスト10-3-BFSとラベルづけを使った増加道の発見方法" data-toc-modified-id="リスト10-3-BFSとラベルづけを使った増加道の発見方法-3.1">リスト10-3 BFSとラベルづけを使った増加道の発見方法</a></span></li><li><span><a href="#リスト10-4--Ford-Fulkersonのアプローチ（デフォルトではEdmonds-Karpのアルゴリズム）" data-toc-modified-id="リスト10-4--Ford-Fulkersonのアプローチ（デフォルトではEdmonds-Karpのアルゴリズム）-3.2">リスト10-4  Ford-Fulkersonのアプローチ（デフォルトではEdmonds-Karpのアルゴリズム）</a></span></li></ul></li><li><span><a href="#10-5-最小コストフローと割り当て問題" data-toc-modified-id="10-5-最小コストフローと割り当て問題-4">10-5 最小コストフローと割り当て問題</a></span><ul class="toc-item"><li><span><a href="#リスト10-5-Bellman-Fordを拡張に使ったBusacker-Gowenのアルゴリズム" data-toc-modified-id="リスト10-5-Bellman-Fordを拡張に使ったBusacker-Gowenのアルゴリズム-4.1">リスト10-5 Bellman-Fordを拡張に使ったBusacker-Gowenのアルゴリズム</a></span></li></ul></li></ul></div>

# 10-1 二部マッチング

## リスト10-1 増加道を使った二部マッチングを求めるアルゴリズム

二部マッチングを求めるアルゴリズムの実装に関しては、クリエイティビティを発揮する余地が存分にあります。実装方法を以下に示します。

In [1]:
# 第5章より
def tr(G):                                      # Gの（すべてのエッジを）反転
    GT = {}
    for u in G:
        GT[u] = set()                           # すべてのノードを取得
    for u in G:
        for v in G[u]:
            GT[v].add(u)                        # すべての反転したエッジを追加
    return GT

In [2]:
from itertools import chain

def match(G, X, Y):                             # 最大二部マッチング
    H = tr(G)                                   # 反転したグラフ
    S, T, M = set(X), set(Y), set()             # 未マッチング（左)、未マッチング（右）、マッチング済み
    while S:                                    # 左側に未マッチングが残っている場合、
        s = S.pop()                             # その1つを抽出
        Q, P = {s}, {}                          # そこから巡回を開始
        while Q:                                # 未訪問のノードが存在する限り巡回
            u = Q.pop()                         # その1つを訪問
            if u in T:                          # 増加道が終了した場合
                T.remove(u)                     # uはマッチング済みとして登録
                break                           # 巡回も終了
            forw = (v for v in G[u] if (u, v) not in M)  # 可能性のある新しいエッジ
            back = (v for v in H[u] if (v, u) in M)      # キャンセリング
            for v in chain(forw, back):         # 出力と入力エッジに沿って
                if v in P:
                    continue                    # 既に訪問済みの場合、無視する
                P[v] = u                        # 巡回用の先行ノード
                Q.add(v)                        # 新たなノードを発見
        while u != s:                           # 増加させ、sまでバックトラック
            u, v = P[u], u                      # 1ステップ分シフト
            if v in G[u]:                       # 前方エッジの場合
                M.add((u, v))                   # 新しいエッジ（プロポーズ）
            else:                               # 後方エッジの場合?
                M.remove((v, u))                # キャンセリング
    return M                                    # マッチングを返す-- エッジの集合

In [3]:
G = {
    0: {2, 3},
    1: {3},
    2: set(),
    3: set()
}

In [4]:
M = match(G, {0, 1}, {2, 3})
sorted(M)
# [(0, 2), (1, 3)]

[(0, 2), (1, 3)]

In [5]:
G = {
    0: {3, 4},
    1: {3, 4},
    2: {4},
    3: set(),
    4: set(),
    5: set()
}
M = match(G, {0, 1, 2}, {3, 4, 5})
sorted(M)
# [(1, 3), (2, 4)]

[(1, 3), (2, 4)]

# 10-2 辺素な道

## リスト10-2 増加道を見つけるための、ラベリング法を使った辺素な道の数え上げ

以下は辺素な道の数え上げアルゴリズムの実装を示しています。前と同様に、`tr`関数はリスト5-11にあります。

In [6]:
def paths(G, s, t):                             # 辺素な道の数え上げ
    H, M, count = tr(G), set(), 0               # 転置, マッチング, カウント
    while True:                                 # 関数が値を返すまで
        Q, P = {s}, {}                          # 巡回のためのキューと木
        while Q:                                # 発見しているが未訪問
            u = Q.pop()                         # 1つのノードを得る
            if u == t:                          # 増加道を発見
                count += 1                      # 道のカウントを増やす
                break                           # 巡回を終了
            forw = (v for v in G[u] if (u, v) not in M)  # 新たなエッジ
            back = (v for v in H[u] if (v, u) in M)      # キャンセリング
            for v in chain(forw, back):         # 出力と入力のエッジに沿って
                if v in P:
                    continue                    # 既に訪問済みの場合、無視
                P[v] = u                        # 巡回用の先行ノード
                Q.add(v)                        # 新たなノードを発見
        else:                                   # tに到達しなかった場合
            return count                        # そこで完了です
        while u != s:                           # 増加させ、sまでバックトラッキング
            u, v = P[u], u                      # 1ステップ分シフトします
            if v in G[u]:                       # 順方向のエッジの場合
                M.add((u, v))                   # 新たなエッジ
            else:                               # 逆方向のエッジの場合
                M.remove((v, u))                # キャンセリング

In [7]:
FF_01_SIMPLE_GRAPH = {
    's': {'u': 1, 'x': 1},
    'u': {'v': 1},
    'v': {'x': 1, 't': 1},
    'x': {'y': 1},
    'y': {'t': 1},
    't': {}
}

In [8]:
G = FF_01_SIMPLE_GRAPH
paths(G, 's', 't')
# 2

2

# 10-3 最大フロー

## リスト10-3 BFSとラベルづけを使った増加道の発見方法

以下、BFSに基づく増加道巡回の実装例を示しています。

In [9]:
from collections import deque, defaultdict
inf = float('inf')


def bfs_aug(G, H, s, t, f):
    P, Q, F = {s: None}, deque([s]), {s: inf}   # 木, キュー, フローラベル

    def label(inc):                             # uからvでフローが増えている場合
        if v in P or inc <= 0:
            return                              # 訪問済み? 到達不可能の場合、無視
        F[v], P[v] = min(F[u], inc), u          # 最大フロー? どの地点から?
        Q.append(v)                             # 発見 -- 後ほど訪問
    while Q:                                    # 発見, 訪問しない
        u = Q.popleft()                         # 1つのノードを取る (FIFO)
        if u == t:
            return P, F[t]                      # tに到達? 増加道!
        for v in G[u]:
            label(G[u][v]-f[u, v])              # 出力エッジに沿ってラベルづけ
        for v in H[u]:
            label(f[v, u])                      # 入力エッジに沿ってラベルづけ
    return None, 0                              # 増加道が見つからなかった場合

In [10]:
def path_from_P(P, s, t):
    res = [t]
    u = t
    while u != s:
        u, v = P[u], u
        res.append(u)
    res.reverse()
    return res

In [11]:
G = {0: {1:1}, 1:{}}
H = tr(G)
f = defaultdict(int)
P, c = bfs_aug(G, H, 0, 1, f)
path_from_P(P, 0, 1)
# [0, 1]

[0, 1]

In [12]:
c
# 1

1

In [13]:
FF_SIMPLE_GRAPH = {
    's': {'u': 2, 'x': 2},
    'u': {'v': 1},
    'v': {'x': 1, 't': 2},
    'x': {'y': 1},
    'y': {'t': 2},
    't': {}
}

In [14]:
G = FF_SIMPLE_GRAPH
H = tr(G)
f = defaultdict(int)
f['s','u'] = 1
f['u','v'] = 1
f['v','x'] = 1
f['x','y'] = 1
f['y','t'] = 1
P, c = bfs_aug(G, H, 's', 't', f)
path_from_P(P, 's', 't')
# ['s', 'x', 'v', 't']

['s', 'x', 'v', 't']

In [15]:
c
# 1

1

## リスト10-4  Ford-Fulkersonのアプローチ（デフォルトではEdmonds-Karpのアルゴリズム）

完全なFord-Fulkersonのアプローチを以下に示します。

In [16]:
from collections import defaultdict


def ford_fulkerson(G, s, t, aug=bfs_aug):       # sからtまでの最大フローを発見する
    H, f = tr(G), defaultdict(int)              # 転置とフロー
    while True:                                 # 改善しながら
        P, c = aug(G, H, s, t, f)               # 増加道と容量・停滞
        if c == 0:
            return f                            # 増加道が見つからなかった場合、終了!
        u = t                                   # 増加開始
        while u != s:                           # sまでバックトラック
            u, v = P[u], u                      # 1ステップ分シフト
            if v in G[u]:
                f[u, v] += c                    # 順方向エッジの場合、スラックを追加
            else:
                f[v, u] -= c                    # 逆方向のエッジの場合、スラックをキャンセル

In [17]:
G = FF_SIMPLE_GRAPH
f = ford_fulkerson(G, 's', 't')
sorted(f.items()) ==  [(('s', 'u'), 1), (('s', 'x'), 1),
(('u', 'v'), 1), (('v', 't'), 1), (('v', 'x'), 0), (('x', 'y'), 1), 
(('y', 't'), 1)]
# True

True

# 10-5 最小コストフローと割り当て問題

## リスト10-5 Bellman-Fordを拡張に使ったBusacker-Gowenのアルゴリズム

以下、Busacker-Gowenのアルゴリズムの実装例です。

In [18]:
def busacker_gowen(G, W, s, t):                 # 最小コスト最大フロー
    def sp_aug(G, H, s, t, f):                  # 最短経路 (Bellman-Fordのアルゴリズム)
        D, P, F = {s: 0}, {s: None}, {s: inf, t: 0}  # 距離, 先行ノード、フロー

        def label(inc, cst):                    # ラベル + 近似
            if inc <= 0:
                return False                    # フローが増えていない場合、スキップ
            d = D.get(u, inf) + cst             # 新たな拡張距離
            if d >= D.get(v, inf):
                return False  # 改善しな場合、スキップ
            D[v], P[v] = d, u                   # 距離と先行ノードを更新
            F[v] = min(F[u], inc)               # フローラベルを更新
            return True                         # 変更あり
        for _ in G:                             # n = len(G) ラウンド
            changed = False                     # 今のところ変化なし
            for u in G:                         # 各fromノード
                for v in G[u]:                  # 各順方向のtoノード
                    changed |= label(G[u][v]-f[u, v], W[u, v])
                for v in H[u]:                  # 各逆方向のtoノード
                    changed |= label(f[v, u], -W[v, u])
            if not changed:
                break                           # 変化がない場合: 終了
        else:                                   # nラウンドで終了しない場合
            raise ValueError('negative cycle')  # 負のサイクルを検知
        return P, F[t]                          # 先行ノードとフローがtに到達
    return ford_fulkerson(G, s, t, sp_aug)      # Bellman-Fordを使った最大フローを返す

In [19]:
G = {
    0: {1: 3, 2: 3},
    1: {3: 2, 4: 2},
    2: {3: 1, 4: 2},
    3: {5: 2},
    4: {5: 2},
    5: {}
}
W = {
    (0, 1): 3,
    (0, 2): 1,
    (1, 3): 1,
    (1, 4): 1,
    (2, 3): 4,
    (2, 4): 2,
    (3, 5): 2,
    (4, 5): 1
}
f1 = ford_fulkerson(G, 0, 5)
for u, v in W:
    assert f1[u, v] <= G[u][v]
f1[3, 5] + f1[4, 5]
# 4

4

In [20]:
f1[0, 1] + f1[0, 2]
# 4

4

In [21]:
f2 = busacker_gowen(G, W, 0, 5)
for u, v in W:
    assert f2[u, v] <= G[u][v]
sum(f2[key]*W[key] for key in W)
# 20

20

In [22]:
fs = [f2[key] for key in sorted(W)]
fs
# [2, 2, 2, 0, 0, 2, 2, 2]

[2, 2, 2, 0, 0, 2, 2, 2]